# Compilation Examples with Routing

In [1]:
from pytket import Circuit
from pytket.circuit.display import view_browser as vb
from pytket.extensions.aqt.backends.aqt_multi_zone import AQTMultiZoneBackend
from pytket.extensions.aqt.multi_zone_architecture.named_architectures import (
    four_zones_in_a_line,
    racetrack,
    racetrack_4_gatezones,
    grid12,
    grid12_mod
)
from pytket.extensions.aqt.multi_zone_architecture.initial_placement.settings import (
    InitialPlacementSettings,
    InitialPlacementAlg,
)
from pytket.extensions.aqt.multi_zone_architecture.circuit_routing.settings import (
    RoutingSettings,
    RoutingAlg,
)
from pytket.extensions.aqt.multi_zone_architecture.compilation_settings import (
    CompilationSettings,
)
import time

First lets define some circuits

In [2]:
## A simple GHZ circuit
ghz_circuit = Circuit(16)
ghz_circuit.H(0)
for i in range(ghz_circuit.n_qubits - 1):
    ghz_circuit.CX(i, i + 1)
ghz_circuit.measure_all()

## Quantum advantage circuit template
def quantum_advantage_circuit(Lx: int, Ly: int) -> Circuit:
    N = Lx * Ly
    Ntrot = 30
    dt = 0.2
    hx = 2
    J = 1

    def c(x, y):
        if (x - y) % N == Lx or (y - x) % N == Lx:
            return 1
        if x // Lx == y // Lx and x == y - (y % Lx) + (((y % Lx) + 1) % Lx):
            return 1
        if x // Lx == y // Lx and y == x - (x % Lx) + (((x % Lx) + 1) % Lx):
            return 1
        return 0

    coupling_list = []
    for i in range(0, N):
        for j in range(i + 1, N):
            if c(i, j) == 1:
                coupling_list.append([i, j])

    U = Circuit(N)

    for t in range(Ntrot):
        n_trotter_steps = t

        if n_trotter_steps == 0:
            for j in range(N):
                U.Rx(-2 * dt * hx / 2, j)

        for coupling in coupling_list:
            U.ZZPhase(-2 * dt * J, coupling[0], coupling[1])
        for j in range(N):
            U.Rx(-2 * dt * hx, j)

        if n_trotter_steps == Ntrot - 1:
            for j in range(N):
                U.Rx(2 * dt * hx / 2, j)
    return U

## 56 qubit quantum advantage circuit
advantage_circuit_56 = quantum_advantage_circuit(7, 8)

## 30 qubit quantum advantage circuit
advantage_circuit_30 = quantum_advantage_circuit(5, 6)

def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(0, n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
    return circ

## 16 qft circuit
qft_circ = build_qft_circuit(16)



Now some MultiZone Backends

In [3]:
line_backend = AQTMultiZoneBackend(
    architecture=four_zones_in_a_line, access_token="invalid"
)

racetrack_backend = AQTMultiZoneBackend(
    architecture=racetrack, access_token="invalid"
)

racetrack_4_gatezones_backend = AQTMultiZoneBackend(
    architecture=racetrack_4_gatezones, access_token="invalid"
)

grid_backend = AQTMultiZoneBackend(
    architecture=grid12, access_token="invalid"
)

grid_mod_backend = AQTMultiZoneBackend(
    architecture=grid12_mod, access_token="invalid"
)


And some different compilation settings

In [4]:
order_init = InitialPlacementSettings(
    algorithm=InitialPlacementAlg.qubit_order,
    zone_free_space=2,
    n_threads=1,
    max_depth=200,
)
graph_routing = RoutingSettings(
    algorithm=RoutingAlg.graph_partition,
    debug_level=0,
    n_threads=1,
)
graph_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=graph_routing,
)

greedy_routing = RoutingSettings(
    algorithm=RoutingAlg.greedy,
)
greedy_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=greedy_routing,
)

## GHZ circuit on a line architecture with two central gate zones and two outer memory zones (8-6-6-8)

In [6]:
ghz_precompiled = line_backend.precompile_circuit(ghz_circuit, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
ghz_graph_routed = line_backend.route_precompiled(ghz_precompiled, graph_compilation_settings)
ghz_greedy_routed = line_backend.route_precompiled(ghz_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", ghz_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", ghz_greedy_routed.get_n_shuttles())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
Shuttles graph:  31
Shuttles greedy:  31


## QFT circuit on a line architecture with two central gate zones and two outer memory zones (8-6-6-8)

In [7]:
qft_precompiled = line_backend.precompile_circuit(qft_circ, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
ghz_graph_routed = line_backend.route_precompiled(qft_precompiled, graph_compilation_settings)
ghz_greedy_routed = line_backend.route_precompiled(qft_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", ghz_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", ghz_greedy_routed.get_n_shuttles())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
Shuttles graph:  181
Shuttles greedy:  781


## 56 qubit "quantum advantage" circuit on all gate-zone "racetrack" architecture

In [8]:

adv_precomp = racetrack_backend.precompile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = racetrack_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = racetrack_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed.get_n_shuttles())

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
Shuttles graph:  13665
Shuttles greedy:  19618


## 56 qubit "quantum advantage" circuit on 4 sequential gate-zone "racetrack" architecture

In [14]:

adv_precomp2 = racetrack_4_gatezones_backend.precompile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp2.n_1qb_gates())
print("#gate depth =", adv_precomp2.depth())
print("#2qb gates =", adv_precomp2.n_2qb_gates())
print("#2qb gate depth =", adv_precomp2.depth_2q())
adv_graph_routed2 = racetrack_4_gatezones_backend.route_precompiled(adv_precomp2, graph_compilation_settings)
adv_greedy_routed2 = racetrack_4_gatezones_backend.route_precompiled(adv_precomp2, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed2.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed2.get_n_shuttles())

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
Shuttles graph:  15062
Shuttles greedy:  28223


## Grid Architecture: All gate zones

In [15]:

adv_precomp = grid_backend.precompile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = grid_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = grid_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed.get_n_shuttles())

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
Shuttles graph:  2192
Shuttles greedy:  2579


## Grid Architecture: Four gate zones

In [16]:
adv_gm_precompiled = grid_mod_backend.precompile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_gm_precompiled.n_1qb_gates())
print("#gate depth =", adv_gm_precompiled.depth())
print("#2qb gates =", adv_gm_precompiled.n_2qb_gates())
print("#2qb gate depth =", adv_gm_precompiled.depth_2q())
adv_gm_graph_routed = grid_mod_backend.route_precompiled(adv_gm_precompiled, graph_compilation_settings)
adv_gm_greedy_routed = grid_mod_backend.route_precompiled(adv_gm_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", adv_gm_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_gm_greedy_routed.get_n_shuttles())

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
Shuttles graph:  3641
Shuttles greedy:  4086
